In [12]:
import os
import time
import glob
import itertools as it
import numpy as np
import cv2
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
%matplotlib inline
%run Functions.py

In [13]:
base_dir = 'test_images/vehicles/'
image_types = os.listdir(base_dir)
cars = []
for image_type in image_types:
    cars.extend(glob.glob(base_dir + image_type + '/*'))
    
base_dir = 'test_images/non-vehicles/'
image_types = os.listdir(base_dir)
not_cars = []
for image_type in image_types:
    not_cars.extend(glob.glob(base_dir + image_type + '/*'))
    
print('Number of vehicles images:', len(cars))
print('Number of non-vehicles images:', len(not_cars))

Number of vehicles images: 8792
Number of non-vehicles images: 8968


In [14]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 32
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32)
hist_bins = 32
spatial_feat = True
hist_feat = True
hog_feat = True
ystart = 400
ystop = 656
y_start_stop = [ystart, ystop]

t=time.time()
car_features = extract_features(cars, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
not_car_features = extract_features(not_cars, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors


X = np.vstack((car_features, not_car_features)).astype(np.float64) 
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.1, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

176.51 Seconds to extract HOG features...
Using: 32 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 21984
45.88 Seconds to train SVC...
Test Accuracy of SVC =  0.9932
My SVC predicts:  [0. 1. 0. 1. 1. 0. 1. 0. 0. 0.]
For these 10 labels:  [0. 1. 0. 1. 1. 0. 1. 0. 0. 0.]
0.05803 Seconds to predict 10 labels with SVC


In [15]:
cls_dict = {'svc': svc, 'X_scaler': X_scaler, 'color_space': color_space, 'orient': orient,
                       'pix_per_cell': pix_per_cell, 'cell_per_block': cell_per_block, 'hog_channel': hog_channel,
                       'spatial_size': spatial_size, 'hist_bins': hist_bins, 'spatial_feat': spatial_feat,
                       'hist_feat': hist_feat, 'hog_feat': hog_feat, 'y_start_stop': y_start_stop}

    # Dump the dictionary to a pickle file for future use
pickle.dump(cls_dict,open('classifier_LinearSVC_YCrCb_991_HogALL_16pixpercell.p', "wb"))

print("Classifier saved.")

Classifier saved.
